In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!git clone https://github.com/HSLCY/GlossBERT.git
%cd GlossBERT

Cloning into 'GlossBERT'...
remote: Enumerating objects: 92, done.
remote: Total 92 (delta 0), reused 0 (delta 0), pack-reused 92 (from 1)
Receiving objects: 100% (92/92), 14.42 MiB | 15.98 MiB/s, done.
Resolving deltas: 100% (37/37), done.
/content/GlossBERT


In [4]:
!bash preparation.sh

./Evaluation_Datasets/senseval2/senseval2.data.xml
./Evaluation_Datasets/senseval3/senseval3.data.xml
./Evaluation_Datasets/semeval2007/semeval2007.data.xml
./Evaluation_Datasets/semeval2013/semeval2013.data.xml
./Evaluation_Datasets/semeval2015/semeval2015.data.xml
./Evaluation_Datasets/ALL/ALL.data.xml
./Training_Corpora/SemCor/semcor.data.xml
./Evaluation_Datasets/senseval2/senseval2.gold.key.txt
./Evaluation_Datasets/senseval2/senseval2.csv
./Evaluation_Datasets/senseval2/senseval2_test_sent_cls.csv
206941
["'s_gravenhage%1:15:00::" 8950407 1 0
 'the site of the royal residence and the de facto capital in the western part of the Netherlands; seat of the International Court of Justice']
157300
1
[('happy%3:00:00::', 'enjoying or showing or marked by joy or pleasure')]
3
[('happy%5:00:00:felicitous:00', 'well expressed and to the point'), ('happy%5:00:00:fortunate:00', 'marked by good fortune'), ('happy%5:00:00:willing:00', 'eagerly disposed to act or to be of service')]
6
[('hard%3:

In [7]:
%cd /content/GlossBERT

/content/GlossBERT


In [11]:

!pip install gdown -q

!gdown --id 1iq_h3zLTflraEU_7tVLnPcVQTeyGDNKE -O Sent_CLS_WS.zip

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1iq_h3zLTflraEU_7tVLnPcVQTeyGDNKE
From (redirected): https://drive.google.com/uc?id=1iq_h3zLTflraEU_7tVLnPcVQTeyGDNKE&confirm=t&uuid=cf918e94-2049-4739-b728-8dea1f7e854e
To: /content/GlossBERT/Sent_CLS_WS.zip
100% 406M/406M [00:07<00:00, 51.5MB/s]


In [12]:
!unzip Sent_CLS_WS.zip -d .

Archive:  Sent_CLS_WS.zip
  inflating: ./config.json           
  inflating: ./pytorch_model.bin     
  inflating: ./vocab.txt             


In [ ]:
%cd /content/GlossBERT

# Make the directory for the model
!mkdir -p Sent_CLS_WS

# Move files into that folder
!mv config.json Sent_CLS_WS/
!mv vocab.txt Sent_CLS_WS/
!mv pytorch_model.bin Sent_CLS_WS/




/content/GlossBERT
mv: cannot stat 'config.json': No such file or directory
mv: cannot stat 'vocab.txt': No such file or directory
mv: cannot stat 'pytorch_model.bin': No such file or directory


In [14]:
!pip install boto3 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 136.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.9 MB/s eta 0:00:00


In [19]:
!python run_infer_demo_sent_cls_ws.py --bert_model ./Sent_CLS_WS

12/08/2025 21:22:16 - INFO - tokenization -   loading vocabulary file ./Sent_CLS_WS/vocab.txt
12/08/2025 21:22:16 - INFO - modeling -   loading archive file ./Sent_CLS_WS
12/08/2025 21:22:16 - INFO - modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

input: U.N. group drafts plan to reduce emissions
lemma: plan
there are 7 candidate senses of "plan"
results:
sense_key: plan%1:09:00::
gloss: a series of steps to be carried out or goals to be accomplished


In [20]:
import torch
import torch.nn.functional as F
from tokenization import BertTokenizer
from modeling import BertForSequenceClassification

class GlossBERTScorer:
    def __init__(self, model_dir="./Sent_CLS_WS", no_cuda=False, max_seq_length=512):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() and not no_cuda else "cpu"
        )
        self.max_seq_length = max_seq_length

        # Sent_CLS_WS is a binary classifier: label 0/1
        self.tokenizer = BertTokenizer.from_pretrained(model_dir, do_lower_case=True)
        self.model = BertForSequenceClassification.from_pretrained(
            model_dir, num_labels=2
        )
        self.model.to(self.device)
        self.model.eval()

    class InputFeatures:
        def __init__(self, input_ids, input_mask, segment_ids):
            self.input_ids = input_ids
            self.input_mask = input_mask
            self.segment_ids = segment_ids

    def _truncate_seq_pair(self, tokens_a, tokens_b, max_length):
        while True:
            total_length = len(tokens_a) + len(tokens_b)
            if total_length <= max_length:
                break
            # kill from the longer one
            if len(tokens_a) > len(tokens_b):
                tokens_a.pop()
            else:
                tokens_b.pop()

    def _mark_target(self, sentence, start_idx, end_idx):
        """
        sentence: string
        start_idx, end_idx: indices over sentence.split(" ")
        """
        tokens = sentence.split(" ")
        target = " ".join(tokens[start_idx:end_idx])
        if len(tokens) > end_idx:
            tokens = (
                tokens[:start_idx]
                + ['"'] + tokens[start_idx:end_idx] + ['"']
                + tokens[end_idx:]
            )
        else:
            tokens = (
                tokens[:start_idx]
                + ['"'] + tokens[start_idx:end_idx] + ['"']
            )
        marked = " ".join(tokens)
        return marked, target

    def _build_features(self, marked_sentence, target_str, gloss_list):
        features = []
        for gloss in gloss_list:
            text_a = marked_sentence
            text_b = f"{target_str} : {gloss}"

            tokens_a = self.tokenizer.tokenize(text_a)
            tokens_b = self.tokenizer.tokenize(text_b)

            self._truncate_seq_pair(tokens_a, tokens_b, self.max_seq_length - 3)

            tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
            segment_ids = [0] * len(tokens)

            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

            input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
            input_mask = [1] * len(input_ids)

            # pad
            pad_len = self.max_seq_length - len(input_ids)
            input_ids += [0] * pad_len
            input_mask += [0] * pad_len
            segment_ids += [0] * pad_len

            features.append(self.InputFeatures(input_ids, input_mask, segment_ids))

        return features

    def score_glosses(self, sentence, start_idx, end_idx, gloss_list):
        """
        sentence: ambiguous sentence (string)
        start_idx, end_idx: token indices (over sentence.split()) of the homonym span
        gloss_list: list[str], e.g. [sense1_text, sense2_text]

        Returns:
            probs_yes: numpy array shape (len(gloss_list),)
                probability of label "1" (sense is correct) for each gloss.
        """
        marked_sentence, target_str = self._mark_target(sentence, start_idx, end_idx)
        eval_features = self._build_features(marked_sentence, target_str, gloss_list)

        input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long).to(self.device)
        input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long).to(self.device)
        segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long).to(self.device)

        with torch.no_grad():
            logits = self.model(
                input_ids=input_ids,
                token_type_ids=segment_ids,
                attention_mask=input_mask,
                labels=None
            )
            probs = torch.softmax(logits, dim=-1).cpu().numpy()

        probs_yes = probs[:, 1]
        return probs_yes

In [30]:
import re

def levenshtein(a, b):
    n, m = len(a), len(b)
    dp = [[0] * (m + 1) for _ in range(n + 1)]
    for i in range(n + 1):
        dp[i][0] = i
    for j in range(m + 1):
        dp[0][j] = j
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost = 0 if a[i - 1] == b[j - 1] else 1
            dp[i][j] = min(
                dp[i - 1][j] + 1,      # deletion
                dp[i][j - 1] + 1,      # insertion
                dp[i - 1][j - 1] + cost  # substitution
            )
    return dp[n][m]


def find_homonym_span(sentence, homonym):
    """
    Return (start_idx, end_idx) of the ambiguous word in `sentence`.
    1. Try exact match.
    2. Try simple singular/plural variants.
    3. Fallback: pick token with minimum Levenshtein distance to `homonym`.
    """
    raw_tokens = sentence.split()
    tokens = [re.sub(r"[^\w]", "", t).lower() for t in raw_tokens]
    hom = homonym.lower()

    # 1) Exact match
    for i, tok in enumerate(tokens):
        if tok == hom:
            return i, i + 1

    # 2) Simple morphology variants (plural -> singular etc.)
    variants = {hom}
    if hom.endswith("ies"):
        variants.add(hom[:-3] + "y")
    if hom.endswith("es"):
        variants.add(hom[:-2])
    if hom.endswith("s") and not hom.endswith("ss"):
        variants.add(hom[:-1])

    for i, tok in enumerate(tokens):
        if tok in variants:
            return i, i + 1

    # 3) Fuzzy fallback: choose closest token by edit distance
    best_i, best_dist = None, float("inf")
    for i, tok in enumerate(tokens):
        if not tok:
            continue
        d = levenshtein(hom, tok)
        if d < best_dist:
            best_dist, best_i = d, i

    # Allow some fuzziness; 3 edits works fine for denominations↔denotations
    max_allowed = max(3, len(hom) // 3 + 1)
    if best_i is not None and best_dist <= max_allowed:
        # Debug print if you want to see what it matched:
        # print(f"[FUZZY MATCH] homonym={homonym}, token={raw_tokens[best_i]}, dist={best_dist}")
        return best_i, best_i + 1

    # If even fuzzy match fails, then something is really off
    raise ValueError(f"Homonym '{homonym}' not found (even fuzzily) in sentence: {sentence}")

In [31]:
sentence = "The potential couldn't be measured."
homonym = "potential"
print(sentence.split(" "))
print(find_homonym_span(sentence, homonym))
# should print something like: (1, 2)

['The', 'potential', "couldn't", 'be', 'measured.']
(1, 2)


In [32]:
gb = GlossBERTScorer("./Sent_CLS_WS")
probs = gb.score_glosses(
    sentence,
    *find_homonym_span(sentence, homonym),
    gloss_list=[
        "the difference in electrical charge between two points in a circuit expressed in volts",
        "latent ability that may be developed"
    ]
)
print(probs)  # 2 numbers, one per gloss

[0.01039284 0.38651744]


In [24]:
import json
from collections import defaultdict

def load_ambistory(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    rows = []
    for k, ex in data.items():
        rows.append({
            "id": k,
            "sample_id": ex["sample_id"],
            "homonym": ex["homonym"],
            "sentence": ex["sentence"],
            "precontext": ex["precontext"],
            "ending": ex["ending"],
            "judged_meaning": ex["judged_meaning"],
            "example_sentence": ex["example_sentence"],
            "average": ex["average"],
            "stdev": ex["stdev"],
        })
    return rows

In [25]:
def group_by_story(rows):
    groups = defaultdict(list)
    for r in rows:
        key = (
            r["precontext"],
            r["sentence"],
            r["ending"],
            r["homonym"]
        )
        groups[key].append(r)
    return groups

In [26]:
train_rows = load_ambistory("/content/train.json")
dev_rows   = load_ambistory("/content/dev.json")

train_groups = group_by_story(train_rows)
dev_groups   = group_by_story(dev_rows)

print("Train story setups:", len(train_groups))
print("Dev story setups:", len(dev_groups))

Train story setups: 1140
Dev story setups: 294


In [28]:
import numpy as np
import math

def extract_features(groups, gb):
    X = []
    y = []
    meta = []

    for (pre, sent, ending, homonym), rows in groups.items():

        # find homonym span
        start_idx, end_idx = find_homonym_span(sent, homonym)

        # build glosses for this story
        glosses = [
            r["judged_meaning"] + " " + r["example_sentence"]
            for r in rows
        ]

        probs = gb.score_glosses(
            sent,
            start_idx,
            end_idx,
            glosses
        )

        # shared features
        p_max = probs.max()
        p_second = np.partition(probs, -2)[-2] if len(probs) > 1 else p_max
        entropy = -(probs * np.log(probs + 1e-12)).sum()

        for i, r in enumerate(rows):
            p_judged = probs[i]
            margin = p_judged - max(p for j,p in enumerate(probs) if j != i)
            rank = int((-probs).argsort().tolist().index(i) + 1)
            has_ending = 1 if r["ending"].strip() else 0

            features = [
                p_judged,
                p_max,
                p_second,
                margin,
                entropy,
                rank,
                has_ending
            ]

            X.append(features)
            y.append(r["average"])
            meta.append(r["sample_id"])

    return np.array(X), np.array(y), meta

In [33]:
gb = GlossBERTScorer("./Sent_CLS_WS")

X_train, y_train, meta_train = extract_features(train_groups, gb)
X_dev, y_dev, meta_dev = extract_features(dev_groups, gb)

print(X_train.shape, X_dev.shape)

(2280, 7) (588, 7)


In [35]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr

# X_train, y_train, X_dev, y_dev already exist

# 1. Choose a range of alphas for Ridge
alphas = np.logspace(-3, 3, 13)  # 0.001 ... 1000

ridge = make_pipeline(
    StandardScaler(),
    RidgeCV(alphas=alphas, scoring="neg_mean_squared_error", cv=None)
)

# 2. Train
ridge.fit(X_train, y_train)

# 3. Inspect the chosen alpha
best_alpha = ridge.named_steps["ridgecv"].alpha_
print(f"Chosen alpha: {best_alpha}")

# 4. Predictions
y_pred_train = ridge.predict(X_train)
y_pred_dev   = ridge.predict(X_dev)

# 5. Metrics
mse_train = mean_squared_error(y_train, y_pred_train)
mse_dev   = mean_squared_error(y_dev,   y_pred_dev)
rho_train, _ = spearmanr(y_train, y_pred_train)
rho_dev,   _ = spearmanr(y_dev,   y_pred_dev)

print(f"Train MSE:      {mse_train:.4f}")
print(f"Dev   MSE:      {mse_dev:.4f}")
print(f"Train Spearman: {rho_train:.44f}")
print(f"Dev   Spearman: {rho_dev:.4f}")

Chosen alpha: 316.2277660168379
Train MSE:      1.2876
Dev   MSE:      1.2882
Train Spearman: 0.29213528199998078926924449660873506218194962
Dev   Spearman: 0.2915


In [38]:
import json
import numpy as np

# y_pred_dev is ridge predictions
# meta_dev should be a list of dicts like {"id": "..."} in the same order as X_dev
# If don't have meta_dev yet, you can build it alongside extract_features.

# 1) Optionally clip predictions to [1, 5] to obey rating scale
y_pred_dev_clipped = np.clip(y_pred_dev, 1.0, 5.0)

pred_path = "dev_predictions_ridge.jsonl"
with open(pred_path, "w", encoding="utf-8") as f:
    for meta, yhat in zip(meta_dev, y_pred_dev_clipped):
        line = {
            "id": str(meta),   # IMPORTANT: must match ids in dev.json
            "prediction": float(yhat)
        }
        f.write(json.dumps(line) + "\n")

print("Wrote predictions to", pred_path)

Wrote predictions to dev_predictions_ridge.jsonl


In [51]:
import json

old_path = "dev_predictions_ridge.jsonl"
new_path = "dev_predictions_ridge_fixed.jsonl"

# Read existing predictions
with open(old_path, "r") as f:
    old_lines = [json.loads(line) for line in f]

print("Old predictions:", len(old_lines))

new_lines = []
for i, item in enumerate(old_lines):
    new_lines.append({
        "id": i,  # reindex 0..587
        "prediction": item["prediction"]
    })

with open(new_path, "w") as f:
    for obj in new_lines:
        f.write(json.dumps(obj) + "\n")

print("Wrote fixed predictions to", new_path)

Old predictions: 588
Wrote fixed predictions to dev_predictions_ridge_fixed.jsonl


In [53]:
import json
import numpy as np

preds = []
with open("dev_predictions_ridge_fixed.jsonl", "r") as f:
    for line in f:
        obj = json.loads(line)
        preds.append(obj["prediction"])

print("min:", min(preds))
print("max:", max(preds))
print("mean:", float(np.mean(preds)))
print("std:", float(np.std(preds)))

min: 2.541788817869273
max: 3.690432419739377
mean: 3.1609919654765095
std: 0.3255431820367677


In [52]:
!python scoring.py /content/GlossBERT/dev_solution.jsonl dev_predictions_ridge_fixed.jsonl results_ridge.txt

Importing...
Starting Scoring script...
Everything looks OK. Evaluating file dev_predictions_ridge_fixed.jsonl on /content/GlossBERT/dev_solution.jsonl
----------
Spearman Correlation: 0.29153580826050257
Spearman p-Value: 5.531367495938446e-13
----------
Accuracy: 0.608843537414966 (358/588)
Results dumped into scores.json successfully.
